<a href="https://colab.research.google.com/github/ketannehra9/Predictive-Analysis-Assisngment-6/blob/main/PAS_Assignment6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install simpy

import simpy
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, HuberRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, explained_variance_score, max_error

def run_sim(env, arr_rate, srv_rate, wait_list):
    server = simpy.Resource(env, capacity=1)

    def process():
        while True:
            yield env.timeout(random.expovariate(arr_rate))
            arrival_time = env.now
            with server.request() as req:
                yield req
                yield env.timeout(random.expovariate(srv_rate))
                wait_list.append(env.now - arrival_time)

    env.process(process())
    env.run(until=100)

data_points = []
print("Generating data...")

for i in range(1000):
    a = random.uniform(0.5, 4.5)
    s = random.uniform(5.0, 10.0)

    env = simpy.Environment()
    waits = []
    run_sim(env, a, s, waits)

    if len(waits) > 0:
        mean_wait = np.mean(waits)
    else:
        mean_wait = 0

    data_points.append([a, s, mean_wait])

df = pd.DataFrame(data_points, columns=['Arrival', 'Service', 'Wait'])
print("Data generation done.")

X = df[['Arrival', 'Service']].values
y = df['Wait'].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

reg_models = {
    "Linear": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "ElasticNet": ElasticNet(),
    "Huber": HuberRegressor(),
    "SVR": SVR(),
    "KNN": KNeighborsRegressor(),
    "Tree": DecisionTreeRegressor(),
    "Forest": RandomForestRegressor(),
    "ExtraTrees": ExtraTreesRegressor(),
    "GradientBoost": GradientBoostingRegressor(),
    "AdaBoost": AdaBoostRegressor()
}

performance = []
print("Training models...")

for name, model in reg_models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    m1 = mean_absolute_error(y_test, y_pred)
    m2 = mean_squared_error(y_test, y_pred)
    m3 = np.sqrt(m2)
    m4 = r2_score(y_test, y_pred)
    m5 = explained_variance_score(y_test, y_pred)
    m6 = max_error(y_test, y_pred)

    performance.append([name, m1, m2, m3, m4, m5, m6])

results = pd.DataFrame(performance, columns=["Model", "MAE", "MSE", "RMSE", "R2", "Var", "MaxErr"])

matrix = results.iloc[:, 1:].values
weights = [0.1, 0.1, 0.25, 0.3, 0.1, 0.15]
dirs = [-1, -1, -1, 1, 1, -1]

sq_sum = np.sqrt(np.sum(matrix**2, axis=0))
norm_matrix = matrix / sq_sum
weighted_matrix = norm_matrix * weights

best_vals = []
worst_vals = []

for i in range(len(dirs)):
    col = weighted_matrix[:, i]
    if dirs[i] == 1:
        best_vals.append(np.max(col))
        worst_vals.append(np.min(col))
    else:
        best_vals.append(np.min(col))
        worst_vals.append(np.max(col))

dist_best = np.sqrt(np.sum((weighted_matrix - best_vals)**2, axis=1))
dist_worst = np.sqrt(np.sum((weighted_matrix - worst_vals)**2, axis=1))

scores = dist_worst / (dist_best + dist_worst)
results['Topsis'] = scores
results = results.sort_values(by='Topsis', ascending=False)

print(results)

plt.figure(figsize=(15, 6))

plt.subplot(1, 2, 1)
sns.barplot(x='Topsis', y='Model', data=results)
plt.title('Model Ranking by TOPSIS')

plt.subplot(1, 2, 2)
best_model_name = results.iloc[0, 0]
best_model = reg_models[best_model_name]
preds = best_model.predict(X_test)

plt.scatter(y_test, preds, alpha=0.6)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linestyle='--')
plt.xlabel('Actual Wait')
plt.ylabel('Predicted Wait')
plt.title(f'Best Model: {best_model_name}')

plt.tight_layout()
plt.show()